# Eksploracja Danych - Projekt
Tomasz Kiljańczyk (136257)

Wojciech Lulek (136280)

# Załadowanie bibliotek i danych

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

data_dir = './data'

In [ ]:
us_trending_df = pd.read_csv(os.path.join(data_dir, 'USData80.csv'), encoding='cp1250', lineterminator='\n')
us_trending_df = us_trending_df.iloc[: , 1:]
us_trending_df = us_trending_df.rename(columns={'description\r': 'description'})
us_trending_df['description'] = us_trending_df['description'].str.rstrip('\r')
us_trending_df

Dislikes nie jest już możliwe do pobrania, więc można je od razu usunąć

In [ ]:
us_trending_df.drop(columns=['dislikes'], axis=1, inplace=True)

# Analiza danych

## Analiza wstępna

In [ ]:
us_trending_df.shape

In [ ]:
us_trending_df.info()

CategoryId zawiera NaNy, trzeba się ich pozbyć

In [ ]:
us_trending_df['categoryId'].replace(np.NaN, 0.0, inplace=True)

CategoryId jest typu float64. Żeby zamienić je na stringa trzeba je zmienić wpierw na int64

In [ ]:
us_trending_df['categoryId'] = us_trending_df['categoryId'].astype(np.int64)

Kolumny opisujące daty są typu object.
Można je zrzutować na typ datetime64.
categoryId należy zrzutować na typ string, ponieważ nie chcemy go traktować jako dane numeryczne.

In [ ]:
us_trending_df = us_trending_df.astype({
    'trending_date': np.datetime64,
    'publishedAt': np.datetime64,
    'categoryId': str
})
us_trending_df.info()

W kolumnie description występują brakujące wartości.
Można by je uzupełnić pustym stringiem albo spacją.

In [ ]:
us_trending_df['description'].replace(np.NaN, '', inplace=True)
us_trending_df.info()

Przy ręcznej analizie znaleziono też tagi `[None]`.
Chyba najlepiej zamienić je na pusty string.

In [ ]:
us_trending_df['tags'].replace('[None]', '', inplace=True)

In [ ]:
us_trending_df.nunique()

W policzonych unikatowych wartościach widać, że unikatowych wartości video_id jest mniej niż ilość wszystkich rzędów w pliku.

Wynika to z faktu, że w danych znajdują się zrzuty listy trending z wielu dni.
Jeden filmik mógł być trending przez parę dni.

## Wizualizacje

In [ ]:
corrMatrix = us_trending_df.corr(method='spearman')
corrMatrix = corrMatrix.round(4)

mask = np.triu(np.ones_like(corrMatrix, dtype=bool))
np.fill_diagonal(mask, False)

fig, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(corrMatrix, mask=mask, annot=True, fmt='g', ax=ax)
plt.show()

Używamy metryki Spearmana, ponieważ to umożliwia odkrycie nieliniowych zależności między cechami.

Zależności większe niż |0.5|:
* view_count:
    * comment_count
    * dislikes
    * likes
* likes:
    * comment_count
    * dislikes
* dislikes:
    * comment_count

Z wypisanych zależności wynika, że im więcej osób ogląda filmik, tym więcej jest komentarzy, polubień i antypolubień.
To jest logiczna konsekwencja wpływu oglądalności na te cechy.

Polubienia można zastąpić cechą stosunek polubień do liczby wyświetleń.

In [ ]:
us_trending_df['likes_to_view_count_ratio'] = us_trending_df['likes'] / us_trending_df['view_count']
us_trending_df['likes_to_view_count_ratio'] = us_trending_df['likes_to_view_count_ratio'].replace(np.inf, 0.0)
us_trending_df['likes_to_view_count_ratio']

In [ ]:
us_trending_df['likes_to_view_count_ratio'].hist(log=True)

Liczbę komentarzy można zamienić na stosunek komentarzy do liczby obejrzeń.
Należy pamiętać, że stosunek ten może być większy niż 1 (bo jeden użytkownik może komentować wiele razy).

In [ ]:
us_trending_df['comment_count_to_view_count_ratio'] = us_trending_df['comment_count'] / us_trending_df['view_count']
us_trending_df['comment_count_to_view_count_ratio'] = us_trending_df['comment_count_to_view_count_ratio'].fillna(0.0)
us_trending_df['comment_count_to_view_count_ratio'] = us_trending_df['comment_count_to_view_count_ratio'].replace(np.inf, 0.0)
us_trending_df['comment_count_to_view_count_ratio']

In [ ]:
us_trending_df['comment_count_to_view_count_ratio'].hist(log=True)

In [ ]:
us_trending_df['view_count'].hist(log=True)

In [ ]:
us_trending_df['comment_count'].hist(log=True)

In [ ]:
us_trending_df['likes'].hist(log=True)

## Analiza kategorii

In [ ]:
import json

with open(os.path.join(data_dir,'US_category_id.json'), mode='r') as file:
    us_category_id_dict = json.loads(file.read())

us_categories = {}
for category in us_category_id_dict['items']:
    us_categories[category['id']] = category['snippet']['title']

us_categories

In [ ]:
len(us_categories)

Z analizy wynika, że używane jest 15 kategorii, a w pliku JSON dostarczone zostało 32 kategorie.
17 kategorii jest nieużywanych.

In [ ]:
us_trending_df['category'] = us_trending_df['categoryId']
us_trending_df['category'] = us_trending_df['category'].apply(us_categories.get).replace(np.NaN, '')
us_trending_df['category']

In [ ]:
category_counts = us_trending_df.groupby(["category"]).size().reset_index(name='counts')
category_counts

In [ ]:
category_counts.plot(x='category', y='counts', kind='bar', logy=True)

## Analiza tytułów

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(strip_accents='unicode', stop_words='english')

In [ ]:
titles_vectorized_csr = vectorizer.fit_transform(us_trending_df['title'])
token_count_sum = titles_vectorized_csr.sum(axis=0).A1
tokens = vectorizer.get_feature_names_out()
df_dict = {'token': tokens, 'count': token_count_sum}
token_count_df = pd.DataFrame(df_dict)
token_count_df

In [ ]:
top_words = token_count_df.sort_values(by=['count'], ascending=False).head(100).reset_index(drop=True)
print(top_words.to_string())

Notatka:
* Można wpisać słowa jako cechy i potem usuwać metodami z wykładu
* Przydałby się ranking informatywności.
* Można użyć słownika z kategoriami

## Analiza tagów

In [ ]:
vectorizer = CountVectorizer(strip_accents='unicode', token_pattern=r'[^|]+')

In [ ]:
tags_vectorized_csr = vectorizer.fit_transform(us_trending_df['tags'])
token_count_sum = tags_vectorized_csr.sum(axis=0).A1
tokens = vectorizer.get_feature_names_out()
df_dict = {'token': tokens, 'count': token_count_sum}
token_count_df = pd.DataFrame(df_dict)
token_count_df

In [ ]:
top_tags = token_count_df.sort_values(by=['count'], ascending=False).head(100).reset_index(drop=True)
print(top_tags.to_string())

# Inne atrybuty

## Nowe atrybuty na podstawie atrybutów tekstowych

In [ ]:
us_trending_df['title_length'] = us_trending_df['title'].str.len()
us_trending_df['title_length'].hist()

In [ ]:
us_trending_df['title_capital_letters'] = us_trending_df['title'].str.count(r'[A-Z]') / us_trending_df['title_length']
us_trending_df['title_capital_letters'].hist()

In [ ]:
us_trending_df['title_punctuaction'] = us_trending_df['title'].str.count(r'[!"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~]') / us_trending_df['title_length']
us_trending_df['title_punctuaction'].hist()

In [ ]:
us_trending_df['description_height'] = us_trending_df['description'].str.count(r'[\r\n]')
us_trending_df['description_height'].hist(log=True)

In [ ]:
us_trending_df['description_length'] = us_trending_df['description'].str.len()
us_trending_df['description_length'].hist()

In [ ]:
us_trending_df['description_link_count'] = us_trending_df['description'].str.count(r'://')
us_trending_df['description_link_count'].hist(log=True)

Notatka: można podzielić linki na kategorie

In [ ]:
us_trending_df['tag_count'] = us_trending_df['tags'].apply(lambda tags: 0 if tags == '' else tags.count('|') + 1)
us_trending_df['tag_count'].hist()

## Nowe atrybuty na podstawie dat

In [ ]:
us_trending_df['publishedAtHour'] = us_trending_df['publishedAt'].dt.hour
us_trending_df['publishedAtHour'].hist()

In [ ]:
us_trending_df['publishedAtDay'] = us_trending_df['publishedAt'].dt.weekday
us_trending_df['publishedAtDay'].hist() 

In [ ]:
us_trending_df['publishedAtMonth'] = us_trending_df['publishedAt'].dt.month
us_trending_df['publishedAtMonth'].hist()

Notatka:
* Warto się przyjrzeć miesiącom publikacji

In [ ]:
from pandas.tseries.holiday import USFederalHolidayCalendar
cal = USFederalHolidayCalendar()
min_date = us_trending_df['publishedAt'].min()
max_date = us_trending_df['publishedAt'].max()

holidays = cal.holidays(start=min_date, end=max_date)
us_trending_df['publishedAtHoliday'] = us_trending_df['publishedAt'].apply(lambda date: date in holidays)
us_trending_df[us_trending_df['publishedAtHoliday']]

Filmiki z listy trending nie były publikowane z święta.

Notatka:
* można spróbować dzień przed/po świętach

Notatka końcowa:
* Warto sprawdzić rozkłady dla różnych kategorii filmików

# Usuwanie kolumn

In [ ]:
us_trending_df.info()

* Nie da się wykorzystać
  * Nieprzetworzonych tagów
  * Nieprzetworzonych opisów
  * Nieprzetworzonych tytułów
  * video_id
  * channelId - niekoniecznie
  * channelTitle - niekoniecznie
  * trending_date, bo niewystępuje w danych nie-trending
  * nieprzetworzone publishedAt

* Można zamienić na coś innego:
  * likes i dislikes można zamienić na ilość reakcji i procent likes

Notatka:
* channelId/Title można użyć do sprawdzania kategorii
* Trending date może mieć znaczenie na etapie uczenia (np. w jakim miesiącu są trending).

In [ ]:
us_trending_clean_df = us_trending_df.copy()
us_trending_clean_df.drop(
    [
     "title",
     "publishedAt",
     "tags",
     "description"
    ], axis=1, inplace=True
)
us_trending_clean_df.info()

In [ ]:
us_trending_clean_df

In [ ]:
corrMatrix = us_trending_clean_df.corr(method='spearman')
corrMatrix = corrMatrix.round(4)

mask = np.triu(np.ones_like(corrMatrix, dtype=bool))
np.fill_diagonal(mask, False)

fig, ax = plt.subplots(figsize=(20, 20))
sns.heatmap(corrMatrix, mask=mask, annot=True, fmt='g', ax=ax)
plt.show()

In [ ]:
us_trending_clean_df.to_csv("./data/stage_1_us_trending.csv", index=False)